In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle
from copy import deepcopy
import random

# disable warnings to ignore overflow error
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# parameters for CSTR
T_0_cstr = 300
V_cstr = 1
k_0_cstr = 8.46*(np.power(10,6))
C_p_cstr = 0.231
rho_L_cstr = 1000
Q_s_cstr = 0.0
F_cstr = 5
E_cstr = 5*(np.power(10,4))
delta_H_cstr = -1.15*(np.power(10,4))
R_cstr = 8.314
C_A0s_cstr = 4

t_final_cstr = 0.005
t_step_cstr = 0.005/36

# parameters for Batch
V_batch = 1
k_0_batch = 8.46*(np.power(10,7))
C_p_batch = 0.231
rho_L_batch = 1000
Q_s_batch = 0
E_batch = 5*(np.power(10,4))
delta_H_batch = -1.15*(np.power(10,4))
R_batch = 8.314

t_final_batch = 0.05
t_step_batch = 0.05/36

# parameters for PFR
k_0_PFR = 8.46 * (np.power(10,6))
C_p_PFR = 0.231
rho_L_PFR = 1000
u_PFR = 2 # volumetric flow rate  (F/A) Superficial velocity
E_by_R_PFR = 5*(np.power(10,4)) / 8.314
delH_term_PFR = -1.15*(np.power(10,4))
U_PFR = 25
Tc_s_PFR = 293
At_PFR = 0.01 # Area for heating rate equation
A_PFR = 0.002  # Area
length_PFR = 1 # total length of reactor
N_PFR = 10     # number of points to discretize the reactor

t_final_PFR = 0.1
t_step_PFR = 0.1/36

# parameters for neural networks
num_step = 36
num_dims = 4

# parameters for Reptile
seed = 0
plot = True
rng = np.random.RandomState(seed)
innerstepsize = 0.02 # stepsize in inner SGD for reptile
innerepochs = 1 # number of epochs of each inner SGD
outerstepsize0 = 0.1 # stepsize of outer optimization, i.e., meta-optimization
niterations = 1500 # number of outer updates; each iteration we sample one task and update on it
epochs = 1 # number of epochs
ntrain = 32 # size of training minibatches (K)
eval_step = 50 # evaluation step
threshold = 10 # threshold to check data correctness

In [3]:
def generate_new_1000(x):
    return x + x * np.random.uniform(-10, 10)

def generate_new_100(x):
    return x + x * np.random.uniform(-1, 1)

def generate_new_5(x):
    return x + x * np.random.uniform(-0.05, 0.05)

def generate_new_Tc_s(x):
    return x + x * np.random.uniform(-0.068259, 1)

def PFR_simulation(u, delH_term, k_0, C_p, rho_L, E_by_R, Tc, U, At, A, t_final, t_step, init_C, init_T, length, N):

    # Method of lines approximates the spatial derivative using finite difference method which reults in a set of coupled ODE

    def method_of_lines_C(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(C) / np.diff(z) - k_0 * np.exp(-E_by_R/T[1:]) * C[1:]    # for first order
        return np.concatenate([[0], D]) #C0 is constant at entrance


    def method_of_lines_T(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(T) / np.diff(z) + (-delH_term/(rho_L*C_p)) * k_0 * np.exp(-E_by_R/T[1:])* C[1:] + (U/(rho_L*C_p*A)) * At * (Tc - T[1:]) # for first order
        return np.concatenate([[0], D]) #T0 is constant at entrance

    z = np.linspace(0, length, N) # discretized length elements

    #initializing arrays
    init_C_A_2_1 = np.zeros(N)
    init_T_2_1 = np.zeros(N)

    init_C_A_2_2 = np.zeros(N)
    init_T_2_2 = np.zeros(N)

    init_C_A_3 = np.zeros(N)
    init_T_3 = np.zeros(N)

    C_A_3 = np.zeros(N)
    T_3 = np.zeros(N)

    dCAdt1 = method_of_lines_C(init_C, init_T)
    dTdt1 = method_of_lines_T(init_C, init_T)

    for i in range(len(init_C)):
        init_C_A_2_1[i] = init_C[i] + dCAdt1[i] * t_step / 2
        init_T_2_1[i] = init_T[i] + dTdt1[i] * t_step / 2

    dCAdt2_1 = method_of_lines_C(init_C_A_2_1, init_T_2_1)
    dTdt2_1 = method_of_lines_T(init_C_A_2_1, init_T_2_1)

    for i in range(len(init_C)):
        init_C_A_2_2[i] = init_C[i] + dCAdt2_1[i] * t_step / 2
        init_T_2_2[i] = init_T[i] + dTdt2_1[i] * t_step / 2

    dCAdt2_2 = method_of_lines_C(init_C_A_2_2, init_T_2_2)
    dTdt2_2 = method_of_lines_T(init_C_A_2_2, init_T_2_2)

    for i in range(len(init_C)):
        init_C_A_3[i] = init_C[i] + dCAdt2_2[i] * t_step / 2
        init_T_3[i] = init_T[i] + dTdt2_2[i] * t_step / 2

    dCAdt3 = method_of_lines_C(init_C_A_3, init_T_3)
    dTdt3 = method_of_lines_T(init_C_A_3, init_T_3)

    dCAdt2 = np.add(dCAdt2_1,dCAdt2_2)
    dCAdt2 = np.divide(dCAdt2,2)

    dTdt2 = np.add(dTdt2_1,dTdt2_2)
    dTdt2 = np.divide(dTdt2,2)

    for i in range(len(init_C)):
        C_A_3[i] = init_C[i] + t_step / 6 * (dCAdt1[i] + 4*dCAdt2[i] + dCAdt3[i])
        T_3[i] = init_T[i] + t_step / 6 * (dTdt1[i] + 4*dTdt2[i] + dTdt3[i])

    return C_A_3 , T_3

def Batch_simulation(V, k_0, E, R, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating Batch using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = -k_0 * np.exp(-E / (R * T)) * C_A
        dTdt = - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 1 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 1 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def to_tensor(x):
    return tf.convert_to_tensor(x, dtype=tf.float32)

def train_on_batch(x, y, model, optimizer):
    x = to_tensor(x)
    y = to_tensor(y)

    with tf.GradientTape() as tape:
        YHat = model(x)
        loss = mse_loss_fn(y, YHat)

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

def predict(x, model):
    x = to_tensor(x)
    return model(x).numpy()

def compute_loss(x, y, model):
    return np.square(predict(x, model) - y).mean()

In [4]:
def gen_cstr(F, V, C_A0s, k_0, E, R, T_0, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        T_0_new = generate_new_100(T_0)
        V_new = generate_new_1000(V)
        F_new = generate_new_1000(F)
        C_A0s_new = generate_new_100(C_A0s)
        Q_s_new = generate_new_1000(Q_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_5(k_0)
        E_new = generate_new_5(E)
        delta_H_new = generate_new_5(delta_H)

        # generating inputs and initial states for CSTR
        u1_list = np.linspace(-3.5, 3.5, 4, endpoint=True)
        u2_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 20, endpoint=True)
        CA_initial = np.linspace(0, 6, 20, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u1_list:
            C_A0 = u1 + C_A0s_new
            for u2 in u2_list:
                Q = u2 + Q_s_new
                for C_A_initial, T_initial in x_deviation:

                    C_A_list, T_list = CSTR_simulation(F_new, V_new, C_A0, k_0_new, E_new, R, T_0_new, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                    if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                        CA0_input.append(u1)
                        Q_input.append(u2)
                        CA_input.append(C_A_initial)
                        T_input.append(T_initial)

                        CA_output.append(C_A_list)
                        T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of CSTR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [5]:
def gen_batch(V, k_0, E, R, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        V_new = generate_new_1000(V)
        Q_s_new = generate_new_1000(Q_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_1000(k_0)
        E_new = generate_new_5(E)
        delta_H_new = generate_new_5(delta_H)

        # generating inputs and initial states for Batch
        u_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 40, endpoint=True)
        CA_initial = np.linspace(0, 6, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u_list:
            Q = u1 + Q_s_new
            for C_A_initial, T_initial in x_deviation:

                C_A_list, T_list = Batch_simulation(V_new, k_0_new, E_new, R, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Q_input.append(u1)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, Q_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of Batch: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [6]:
def gen_pfr(u, delH_term, k_0, C_p, rho_L, E_by_R, U, At, A, Tc_s, length, N, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        u_new = generate_new_100(u)
        U_new = generate_new_100(U)
        A_new = generate_new_1000(A)
        At_new = generate_new_1000(At)
        Tc_s_new = generate_new_Tc_s(Tc_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_5(k_0)
        E_by_R_new = generate_new_5(E_by_R)
        delH_term_new = generate_new_5(delH_term)

        # generating inputs and initial states for CSTR, all expressed in deviation form
        u_list = np.linspace(100, 300, 4, endpoint=True)
        T_initial = np.linspace(300, 500, 40, endpoint=True)
        CA_initial = np.linspace(0.5, 3, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for physics-informed model
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        Tc_input = list()
        CA0_input = list()

        for u2 in u_list:
            Tc = u2 + Tc_s_new

            for C_A_initial, T_initial in x_deviation:

                z = np.linspace(0, length, N) # discretized length elements

                init_C = np.zeros(N)    # Concentration in reactor at t = 0
                init_C[0] = C_A_initial          # concentration at entrance
                init_T = np.zeros(N)    # T in reactor at t = 0
                for i in range(len(init_T)):
                    if i == 0:
                        init_T[i] = T_initial
                    else:
                        init_T[i] = Tc_s_new

                C_A_list = list()
                T_list = list()

                for i in range(int(t_final / t_step)):

                    CA_next, T_next = PFR_simulation(u_new, delH_term_new, k_0_new, C_p_new, rho_L_new, E_by_R_new, Tc, U_new, At_new, A_new, t_final, t_step, init_C, init_T, length, N)
                    if i % 1 == 0:
                        C_A_list.append(CA_next)
                        T_list.append(T_next)
                    init_C = CA_next
                    init_T = T_next
                if any(abs(i) < 0.001 for i in np.array(T_list)[:,1]) == False and any(abs(i) < 0.001 for i in np.array(C_A_list)[:,1]) == False and any(abs(i) > 10000 for i in np.array(T_list)[:,1]) == False and any(abs(i) > 10000 for i in np.array(C_A_list)[:,1]) == False and any(abs(i) == 0 for i in np.array(T_list)[:,1]) == False and any(abs(i) == 0 for i in np.array(C_A_list)[:,1]) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Tc_input.append(u2)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            CA_output = np.array(CA_output)[:,:,1]
            T_output = np.array(T_output)[:,:,1]

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Tc_input = np.array(Tc_input)
            Tc_input = Tc_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Tc_input, Tc_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of PFR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [7]:
class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Necessary to create the model's state.
# The model doesn't have a state until it's called at least once.
_ = model(tf.zeros((ntrain, num_step, num_dims)))

optimizer = tf.keras.optimizers.Adam()
mse_loss_fn = tf.keras.losses.MeanSquaredError()

In [8]:
# generate testing cstr
isOverflow = True
while isOverflow == True:
    try:
        x_test_cstr, y_test_cstr = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_cstr), size=1)
xtest_plot_1shot_cstr = x_test_cstr[sample_idx]
ytest_plot_1shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=5)
xtest_plot_5shot_cstr = x_test_cstr[sample_idx]
ytest_plot_5shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=10)
xtest_plot_10shot_cstr = x_test_cstr[sample_idx]
ytest_plot_10shot_cstr = y_test_cstr[sample_idx]

# generate testing batch
isOverflow = True
while isOverflow == True:
    try:
        x_test_batch, y_test_batch = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_batch), size=1)
xtest_plot_1shot_batch = x_test_batch[sample_idx]
ytest_plot_1shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=5)
xtest_plot_5shot_batch = x_test_batch[sample_idx]
ytest_plot_5shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=10)
xtest_plot_10shot_batch = x_test_batch[sample_idx]
ytest_plot_10shot_batch = y_test_batch[sample_idx]

# generate testing pfr
isOverflow = True
while isOverflow == True:
    try:
        x_test_pfr, y_test_pfr = gen_pfr(u_PFR, delH_term_PFR, k_0_PFR, C_p_PFR, rho_L_PFR, E_by_R_PFR, U_PFR, At_PFR, A_PFR, Tc_s_PFR, length_PFR, N_PFR, t_final_PFR, t_step_PFR, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_pfr), size=1)
xtest_plot_1shot_pfr = x_test_pfr[sample_idx]
ytest_plot_1shot_pfr = y_test_pfr[sample_idx]

sample_idx = rng.choice(len(x_test_pfr), size=5)
xtest_plot_5shot_pfr = x_test_pfr[sample_idx]
ytest_plot_5shot_pfr = y_test_pfr[sample_idx]

sample_idx = rng.choice(len(x_test_pfr), size=10)
xtest_plot_10shot_pfr = x_test_pfr[sample_idx]
ytest_plot_10shot_pfr = y_test_pfr[sample_idx]

Number of training samples of CSTR:  6240
Number of training samples of Batch:  5572
Number of training samples of PFR:  6400


In [14]:
training_loss = np.zeros(niterations)
test_loss_1shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_pfr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_pfr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_pfr = np.zeros(int(niterations/eval_step+1))

reactor = 0
count = 0

# reptile
for iteration in range(niterations):

    print("Begin iteration ", iteration)

    # begin reptile
    weights_before = deepcopy(model.get_weights())

    # generate task
    isOverflow = True
    while isOverflow == True:
        try:
            # switch to another reactor, 0 for cstr, 1 for batch, 2 for pfr
            if reactor == 0:
                x_all, y_all = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step, num_dims)
                reactor = 1
            elif reactor == 1:
                x_all, y_all = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step, num_dims)
                reactor = 2
            elif reactor == 2:
                x_all, y_all = gen_pfr(u_PFR, delH_term_PFR, k_0_PFR, C_p_PFR, rho_L_PFR, E_by_R_PFR, U_PFR, At_PFR, A_PFR, Tc_s_PFR, length_PFR, N_PFR, t_final_PFR, t_step_PFR, num_step, num_dims)
                reactor = 0
            isOverflow = False
        except ValueError:
            pass

    inds = rng.permutation(len(x_all))

    # for some k number of iterations perform optimization on the task
    for k in range(innerepochs):

        for start in range(0, len(x_all), ntrain):
            mbinds = inds[start:start+ntrain]
            train_on_batch(x_all[mbinds], y_all[mbinds], model, optimizer)

    weights_after = model.get_weights()

    outerstepsize = outerstepsize0 * (1 - iteration / niterations) # linear schedule

    model.set_weights([ weights_before[i] + (weights_after[i] - weights_before[i]) * outerstepsize for i in range(len(model.weights))])

    training_loss[iteration] = compute_loss(x_all, y_all, model)
    print("Training loss: ", training_loss[iteration])

    # reset weights and terminate for NaN training loss
    if np.isnan(training_loss[iteration]) == True:
        model.set_weights(weights_before)
        break

    # begin evaluation
    if plot and iteration==0 or (iteration+1) % eval_step == 0:
        # 1 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_cstr, ytest_plot_1shot_cstr, model, optimizer)

        test_loss_1shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 1-shot CSTR: ", test_loss_1shot_cstr[count])
        model.set_weights(weights_before)

        # 5 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_cstr, ytest_plot_5shot_cstr, model, optimizer)

        test_loss_5shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 5-shots CSTR: ", test_loss_5shot_cstr[count])
        model.set_weights(weights_before)

        # 10 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_cstr, ytest_plot_10shot_cstr, model, optimizer)

        test_loss_10shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 10-shots CSTR: ", test_loss_10shot_cstr[count])
        model.set_weights(weights_before)

        # 1 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_batch, ytest_plot_1shot_batch, model, optimizer)

        test_loss_1shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 1-shot Batch: ", test_loss_1shot_batch[count])
        model.set_weights(weights_before)

        # 5 shot learning for bacth
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_batch, ytest_plot_5shot_batch, model, optimizer)

        test_loss_5shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 5-shots Batch: ", test_loss_5shot_batch[count])
        model.set_weights(weights_before)

        # 10 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_batch, ytest_plot_10shot_batch, model, optimizer)

        test_loss_10shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 10-shots Batch: ", test_loss_10shot_batch[count])
        model.set_weights(weights_before)

        # 1 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_pfr, ytest_plot_1shot_pfr, model, optimizer)

        test_loss_1shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 1-shot PFR: ", test_loss_1shot_pfr[count])
        model.set_weights(weights_before)

        # 5 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_pfr, ytest_plot_5shot_pfr, model, optimizer)

        test_loss_5shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 5-shots PFR: ", test_loss_5shot_pfr[count])
        model.set_weights(weights_before)

        # 10 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_pfr, ytest_plot_10shot_pfr, model, optimizer)

        test_loss_10shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 10-shots PFR: ", test_loss_10shot_pfr[count])
        model.set_weights(weights_before)

        count = count + 1

    if (iteration+1) % 100 == 0:
        filename = 'model_reptile_cstr_batch_pfr_36timesteps.sav'
        pickle.dump(model, open(filename, 'wb'))

Begin iteration  0
Number of training samples of CSTR:  6092
Training loss:  0.2011559675960317
Test loss for 1-shot CSTR:  0.26193288663786873
Test loss for 5-shots CSTR:  0.12087597307294295
Test loss for 10-shots CSTR:  0.1953548936206684
Test loss for 1-shot Batch:  1.6584881437155035
Test loss for 5-shots Batch:  0.889109559210485
Test loss for 10-shots Batch:  1.1371753111331002
Test loss for 1-shot PFR:  0.5085404764969492
Test loss for 5-shots PFR:  0.3986845617314415
Test loss for 10-shots PFR:  0.7455372721378279
Begin iteration  1
Number of training samples of Batch:  5676
Training loss:  1.4411075500370247
Begin iteration  2
Number of training samples of PFR:  6400
Training loss:  0.3903669552703651
Begin iteration  3
Number of training samples of CSTR:  6224
Training loss:  0.24893477786207527
Begin iteration  4
Number of training samples of Batch:  5314
Training loss:  1.1364651691107923
Begin iteration  5
Number of training samples of PFR:  6400
Training loss:  0.4497190

Training loss:  0.5405302319235528
Begin iteration  76
Number of training samples of Batch:  5400
Training loss:  0.8352832006718605
Begin iteration  77
Number of training samples of PFR:  6400
Training loss:  0.4919143592231426
Begin iteration  78
Number of training samples of CSTR:  6096
Training loss:  0.5301037216952782
Begin iteration  79
Number of training samples of Batch:  5134
Training loss:  0.8128065344937393
Begin iteration  80
Number of training samples of PFR:  6400
Training loss:  0.4926677933677879
Begin iteration  81
Number of training samples of CSTR:  6232
Training loss:  0.4522071603630711
Begin iteration  82
Number of training samples of Batch:  5715
Training loss:  0.7073029538760899
Begin iteration  83
Number of training samples of PFR:  6400
Training loss:  0.6850855214125301
Begin iteration  84
Number of training samples of CSTR:  5952
Training loss:  0.505674549781071
Begin iteration  85
Number of training samples of Batch:  6023
Training loss:  0.769983112798

Training loss:  0.3661915266568188
Begin iteration  151
Number of training samples of Batch:  6240
Training loss:  0.4976787570055563
Begin iteration  152
Number of training samples of PFR:  6400
Training loss:  0.39030697246412704
Begin iteration  153
Number of training samples of CSTR:  6286
Training loss:  0.4794765425523771
Begin iteration  154
Number of training samples of Batch:  5287
Training loss:  0.757005118796893
Begin iteration  155
Number of training samples of PFR:  6400
Training loss:  0.42814758602607483
Begin iteration  156
Number of training samples of CSTR:  6355
Training loss:  0.454787713760983
Begin iteration  157
Number of training samples of Batch:  5006
Training loss:  0.7864946502328387
Begin iteration  158
Number of training samples of PFR:  6400
Training loss:  0.5596488614760037
Begin iteration  159
Number of training samples of CSTR:  6048
Training loss:  0.47793008438711193
Begin iteration  160
Number of training samples of Batch:  6240
Training loss:  0.

Number of training samples of PFR:  6400
Training loss:  0.4654896781211881
Begin iteration  231
Number of training samples of CSTR:  6400
Training loss:  0.33155104074503927
Begin iteration  232
Number of training samples of Batch:  6240
Training loss:  0.36810044058365526
Begin iteration  233
Number of training samples of PFR:  6400
Training loss:  0.35018432042365427
Begin iteration  234
Number of training samples of CSTR:  6244
Training loss:  0.3506712714935337
Begin iteration  235
Number of training samples of Batch:  5397
Training loss:  0.7330378360586444
Begin iteration  236
Number of training samples of PFR:  6400
Training loss:  0.4713697541229165
Begin iteration  237
Number of training samples of CSTR:  6400
Training loss:  0.30907481875149306
Begin iteration  238
Number of training samples of Batch:  5188
Training loss:  0.6865471241633243
Begin iteration  239
Number of training samples of PFR:  6400
Training loss:  0.35154533823479683
Begin iteration  240
Number of traini

Number of training samples of PFR:  6400
Training loss:  0.5041283200586528
Begin iteration  306
Number of training samples of CSTR:  6304
Training loss:  0.3638674845118874
Begin iteration  307
Number of training samples of Batch:  6222
Training loss:  0.6320808137113564
Begin iteration  308
Number of training samples of PFR:  6400
Training loss:  0.3142672849706232
Begin iteration  309
Number of training samples of CSTR:  6336
Training loss:  0.36118970070711165
Begin iteration  310
Number of training samples of Batch:  5976
Training loss:  0.18684242620613173
Begin iteration  311
Number of training samples of PFR:  6400
Training loss:  0.3625971936071569
Begin iteration  312
Number of training samples of CSTR:  6274
Training loss:  0.23080219092134618
Begin iteration  313
Number of training samples of Batch:  6240
Training loss:  0.5903416352325809
Begin iteration  314
Number of training samples of PFR:  6400
Training loss:  0.5357061197646729
Begin iteration  315
Number of training

...

Test loss for 1-shot PFR:  0.2030901205109444
Test loss for 5-shots PFR:  0.031157762177158307
Test loss for 10-shots PFR:  0.055214937741733604
Begin iteration  1150
Number of training samples of Batch:  6231
Training loss:  0.6724008607953292
Begin iteration  1151
Number of training samples of PFR:  6400
Training loss:  0.4664029853016686
Begin iteration  1152
Number of training samples of CSTR:  6067
Training loss:  0.31803709496793636
Begin iteration  1153
Number of training samples of Batch:  5550
Training loss:  0.7103448233591088
Begin iteration  1154
Number of training samples of PFR:  6400
Training loss:  0.43255473867027633
Begin iteration  1155
Number of training samples of CSTR:  6400
Training loss:  0.25775904963801527
Begin iteration  1156
Number of training samples of Batch:  5351
Training loss:  0.7314982632584144
Begin iteration  1157
Number of training samples of PFR:  6400
Training loss:  0.40157902063197903
Begin iteration  1158
Number of training samples of CSTR:  

Number of training samples of CSTR:  6160
Training loss:  0.22153268167003698
Begin iteration  1228
Number of training samples of Batch:  5681
Training loss:  0.9399596095198566
Begin iteration  1229
Number of training samples of PFR:  6400
Training loss:  0.36759476566518884
Begin iteration  1230
Number of training samples of CSTR:  6332
Training loss:  0.2674792089934084
Begin iteration  1231
Number of training samples of Batch:  5774
Training loss:  0.8497017303981306
Begin iteration  1232
Number of training samples of PFR:  6400
Training loss:  0.42003802656084993
Begin iteration  1233
Number of training samples of CSTR:  6020
Training loss:  0.3203920981417396
Begin iteration  1234
Number of training samples of Batch:  5831
Training loss:  0.7926462268604221
Begin iteration  1235
Number of training samples of PFR:  6400
Training loss:  0.4535326287453422
Begin iteration  1236
Number of training samples of CSTR:  6149
Training loss:  0.267272233806091
Begin iteration  1237
Number o

Number of training samples of PFR:  6400
Training loss:  0.3841111756288933
Begin iteration  1302
Number of training samples of CSTR:  6304
Training loss:  0.28290785289296755
Begin iteration  1303
Number of training samples of Batch:  5982
Training loss:  0.7523953063529596
Begin iteration  1304
Number of training samples of PFR:  6400
Training loss:  0.5256599063739125
Begin iteration  1305
Number of training samples of CSTR:  6195
Training loss:  0.22675548438664517
Begin iteration  1306
Number of training samples of Batch:  5662
Training loss:  0.6746644166949014
Begin iteration  1307
Number of training samples of PFR:  6400
Training loss:  0.4853740315264506
Begin iteration  1308
Number of training samples of CSTR:  6192
Training loss:  0.316014397437701
Begin iteration  1309
Number of training samples of Batch:  5524
Training loss:  0.656949305500512
Begin iteration  1310
Number of training samples of PFR:  6400
Training loss:  0.6920290296412991
Begin iteration  1311
Number of t

Training loss:  0.3674808857422748
Begin iteration  1380
Number of training samples of CSTR:  5948
Training loss:  0.3619858694417677
Begin iteration  1381
Number of training samples of Batch:  6240
Training loss:  0.4793224817890848
Begin iteration  1382
Number of training samples of PFR:  6400
Training loss:  0.4068336357592663
Begin iteration  1383
Number of training samples of CSTR:  6298
Training loss:  0.2929902866553475
Begin iteration  1384
Number of training samples of Batch:  6030
Training loss:  0.9052168987985421
Begin iteration  1385
Number of training samples of PFR:  6400
Training loss:  0.3572605842767198
Begin iteration  1386
Number of training samples of CSTR:  6299
Training loss:  0.22326617971555587
Begin iteration  1387
Number of training samples of Batch:  5538
Training loss:  0.7955507678184368
Begin iteration  1388
Number of training samples of PFR:  6400
Training loss:  0.37643724108971965
Begin iteration  1389
Number of training samples of CSTR:  6261
Training

Training loss:  0.8321233590169246
Begin iteration  1454
Number of training samples of PFR:  6400
Training loss:  0.40870352753324535
Begin iteration  1455
Number of training samples of CSTR:  6320
Training loss:  0.20759865112130896
Begin iteration  1456
Number of training samples of Batch:  5097
Training loss:  0.8507616834643991
Begin iteration  1457
Number of training samples of PFR:  6400
Training loss:  0.3582630374766042
Begin iteration  1458
Number of training samples of CSTR:  6400
Training loss:  0.2496047349737483
Begin iteration  1459
Number of training samples of Batch:  5089
Training loss:  0.8546602599562388
Begin iteration  1460
Number of training samples of PFR:  6400
Training loss:  0.37133621001867756
Begin iteration  1461
Number of training samples of CSTR:  6304
Training loss:  0.266339827077533
Begin iteration  1462
Number of training samples of Batch:  6240
Training loss:  0.8769759923600399
Begin iteration  1463
Number of training samples of PFR:  6400
Training 

In [15]:
filename = 'model_reptile_cstr_batch_pfr_36timesteps.sav'
pickle.dump(model, open(filename, 'wb'))
# model = pickle.load(open(filename, 'rb'))

In [16]:
np.savetxt("training_reptile_cstr_batch_pfr.txt", training_loss, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_reptile_cstr_cstr_batch_pfr.txt", test_loss_1shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_reptile_cstr_cstr_batch_pfr.txt", test_loss_5shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_reptile_cstr_cstr_batch_pfr.txt", test_loss_10shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_reptile_batch_cstr_batch_pfr.txt", test_loss_1shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_reptile_batch_cstr_batch_pfr.txt", test_loss_5shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_reptile_batch_cstr_batch_pfr.txt", test_loss_10shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_reptile_pfr_cstr_batch_pfr.txt", test_loss_1shot_pfr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_reptile_pfr_cstr_batch_pfr.txt", test_loss_5shot_pfr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_reptile_pfr_cstr_batch_pfr.txt", test_loss_10shot_pfr, fmt='%f', delimiter=" ")